In [1]:
#Import GYM stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#Import helpers
import numpy as np
import random
import os
import pandas as pd

# Import stable baselines stuff
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
def get_data(start = '2017-01-01 00:00:00', end = '2017-12-12 23:55:00'):

    # import standard load profiles

    slp = pd.read_csv('df_p.csv', index_col=0, parse_dates=True)['0'] / 1000
    slp = slp.resample('15min').mean() * 3

    pv = pd.read_csv('Solar_Data-2011.csv', delimiter=';',
                    index_col=0, parse_dates=False)["Generation"] * 3
    pv.index = slp.index
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    
    return slp[start:end], pv[start:end]

In [3]:
class CostEnv(Env):
    def __init__(self):
        # Actions we can take increase in cost, lowering of cost
        self.action_space = Discrete(2)    
        ### Get input data, just choose one day for now
        self.load, self.pv = get_data(start = '2017-01-01 00:00:00', end = '2017-12-12 23:55:00')
        self.pv_price = 0.1
        self.grid_price = 0.4
        self.line_max = 15
        self.grid_penalty = 200
        self.battery_max = 20
        self.battery_state = 0
        ###
        
        # Set episode length
        self.episode_length = len(self.load)
        
        self.observation_space = Dict(
            {
                "load": Box(0, self.episode_length - 1, shape=(2,), dtype=int),
                #"pv": Box(0, self.episode_length - 1, shape=(2,), dtype=int),
            }
        )
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 decrease in cost
        # 1 -1 = 0 Increase in cost
        # self.state += action -1 
        # Reduce episode length by 1 second
        # self.episode_length -= 1 ###Move to the end of the action
        ### We calculate the reward based on the price for the electricity, lower price, "higher" reward
        if action == 0:
            # Take all electricity from grid
            if self.load[len(self.load)-self.episode_length] > self.line_max:
                reward = self.load[len(self.load)-self.episode_length] * self.grid_price * -1 - abs(self.load[len(self.load)-self.episode_length] - self.line_max) * self.grid_penalty
            else:
                reward = self.load[len(self.load)-self.episode_length] * self.grid_price * -1
            
        elif action == 1:
            # Take all electricity from pv
            if self.pv[len(self.pv)-self.episode_length] >= self.load[len(self.load)-self.episode_length]:
                if abs(self.pv[len(self.pv)-self.episode_length] - self.load[len(self.load)-self.episode_length]) > self.line_max:
                    reward = self.load[len(self.load)-self.episode_length] * self.pv_price * -1 - (self.pv[len(self.pv)-self.episode_length] - self.load[len(self.load)-self.episode_length]) * self.grid_penalty
                else:
                    reward = self.load[len(self.load)-self.episode_length] * self.pv_price * -1
            else:
                if (self.load[len(self.load)-self.episode_length] - self.pv[len(self.pv)-self.episode_length]) > self.line_max:
                    reward = self.pv[len(self.pv)-self.episode_length] * self.pv_price * -1 - (self.load[len(self.load)-self.episode_length] - self.pv[len(self.pv)-self.episode_length]) * self.grid_price - (self.load[len(self.load)-self.episode_length] - self.pv[len(self.pv)-self.episode_length]) * self.grid_penalty
                else:
                    reward = self.pv[len(self.pv)-self.episode_length] * self.pv_price * -1
            ### This may lead the agent to always choose action 1, 
            ###because it will always supply the demand and will always be cheaper.
        
        else:
            # Invalid action
            raise(f'Invalid action: {action}')

        # Apply cost fluctuation
        # self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        ### Observation
        observation = {
            "load": self.load[len(self.load)-self.episode_length],
            "pv": self.pv[len(self.pv)-self.episode_length],
            }
        
        ### Either here or before checking self.episode_length
        self.episode_length -= 1
        ### Check if timeseries is over
        if self.episode_length <= 0: 
            done = True
        else:
            done = False
        
        # Return step information
        # return self.state, reward, done, info
        return observation, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.done=False
        # Set episode length
        self.episode_length = len(self.load)
        
        observation = {
            "load": 0, #self.load[len(self.load)-self.episode_length],
            "pv": 0, #self.pv[len(self.pv)-self.episode_length],
            }
        
        return observation





In [25]:
env = CostEnv()

In [26]:
episodes = 20
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        # n_state, reward, done, info = env.step(action)
        observation, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


NotImplementedError: 

In [ ]:
log_path = os.path.join('Training', 'Logs')

model = A2C("MultiInputPolicy", env, verbose=1, tensorboard_log=log_path)

model.learn(total_timesteps=500000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Training\Logs\A2C_5
------------------------------------
| time/                 |          |
|    fps                | 14       |
|    iterations         | 100      |
|    time_elapsed       | 34       |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.134   |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.189   |
|    value_loss         | 0.0358   |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 27       |
|    iterations         | 200      |
|    time_elapsed       | 36       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -0.00771 |
|    explained_variance | -3.53    |
|    learning_rate      

-------------------------------------
| time/                 |           |
|    fps                | 132       |
|    iterations         | 1700      |
|    time_elapsed       | 64        |
|    total_timesteps    | 8500      |
| train/                |           |
|    entropy_loss       | -0.00096  |
|    explained_variance | -0.219    |
|    learning_rate      | 0.0007    |
|    n_updates          | 1699      |
|    policy_loss        | -2.05e-05 |
|    value_loss         | 0.0383    |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 136       |
|    iterations         | 1800      |
|    time_elapsed       | 65        |
|    total_timesteps    | 9000      |
| train/                |           |
|    entropy_loss       | -0.00197  |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 1799      |
|    policy_loss        | -2.16e-05 |
|    value_l

------------------------------------
| time/                 |          |
|    fps                | 172      |
|    iterations         | 3300     |
|    time_elapsed       | 95       |
|    total_timesteps    | 16500    |
| train/                |          |
|    entropy_loss       | -0.00052 |
|    explained_variance | -10.8    |
|    learning_rate      | 0.0007   |
|    n_updates          | 3299     |
|    policy_loss        | 2.87e-06 |
|    value_loss         | 0.0513   |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 173      |
|    iterations         | 3400     |
|    time_elapsed       | 97       |
|    total_timesteps    | 17000    |
| train/                |          |
|    entropy_loss       | -0.00125 |
|    explained_variance | -1.45    |
|    learning_rate      | 0.0007   |
|    n_updates          | 3399     |
|    policy_loss        | 8.85e-06 |
|    value_loss         | 0.00574  |
-

-------------------------------------
| time/                 |           |
|    fps                | 188       |
|    iterations         | 4900      |
|    time_elapsed       | 129       |
|    total_timesteps    | 24500     |
| train/                |           |
|    entropy_loss       | -0.000727 |
|    explained_variance | -2.38e-07 |
|    learning_rate      | 0.0007    |
|    n_updates          | 4899      |
|    policy_loss        | 3.74e-06  |
|    value_loss         | 0.00361   |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 188       |
|    iterations         | 5000      |
|    time_elapsed       | 132       |
|    total_timesteps    | 25000     |
| train/                |           |
|    entropy_loss       | -0.000781 |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 4999      |
|    policy_loss        | 3.46e-06  |
|    value_l

-------------------------------------
| time/                 |           |
|    fps                | 199       |
|    iterations         | 6500      |
|    time_elapsed       | 162       |
|    total_timesteps    | 32500     |
| train/                |           |
|    entropy_loss       | -0.000407 |
|    explained_variance | 1.19e-07  |
|    learning_rate      | 0.0007    |
|    n_updates          | 6499      |
|    policy_loss        | -4.67e-06 |
|    value_loss         | 0.0199    |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 200       |
|    iterations         | 6600      |
|    time_elapsed       | 164       |
|    total_timesteps    | 33000     |
| train/                |           |
|    entropy_loss       | -5.12e-05 |
|    explained_variance | -6.75     |
|    learning_rate      | 0.0007    |
|    n_updates          | 6599      |
|    policy_loss        | 2.29e-08  |
|    value_l

...

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 267       |
|    iterations         | 36500     |
|    time_elapsed       | 682       |
|    total_timesteps    | 182500    |
| train/                |           |
|    entropy_loss       | -4.43e-05 |
|    explained_variance | -1.08     |
|    learning_rate      | 0.0007    |
|    n_updates          | 36499     |
|    policy_loss        | 2.28e-07  |
|    value_loss         | 0.0129    |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 267       |
|    iterations         | 36600     |
|    time_elapsed       | 684       |
|    total_timesteps    | 183000    |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 266       |
|    iterations         | 37800     |
|    time_elapsed       | 710       |
|    total_timesteps    | 189000    |
| train/                |           |
|    entropy_loss       | -3.01e-05 |
|    explained_variance | 0.488     |
|    learning_rate      | 0.0007    |
|    n_updates          | 37799     |
|    policy_loss        | -9.92e-08 |
|    value_loss         | 0.0118    |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 266       |
|    iterations         | 37900     |
|    time_elapsed       | 712       |
|    total_timesteps    | 189500    |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 265       |
|    iterations         | 39100     |
|    time_elapsed       | 736       |
|    total_timesteps    | 195500    |
| train/                |           |
|    entropy_loss       | -2.97e-05 |
|    explained_variance | -0.451    |
|    learning_rate      | 0.0007    |
|    n_updates          | 39099     |
|    policy_loss        | -1.05e-07 |
|    value_loss         | 0.00364   |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 265       |
|    iterations         | 39200     |
|    time_elapsed       | 738       |
|    total_timesteps    | 196000    |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 264       |
|    iterations         | 40400     |
|    time_elapsed       | 762       |
|    total_timesteps    | 202000    |
| train/                |           |
|    entropy_loss       | -6.07e-05 |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 40399     |
|    policy_loss        | 1.15e-07  |
|    value_loss         | 0.000874  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 264       |
|    iterations         | 40500     |
|    time_elapsed       | 764       |
|    total_timesteps    | 202500    |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 264       |
|    iterations         | 41700     |
|    time_elapsed       | 788       |
|    total_timesteps    | 208500    |
| train/                |           |
|    entropy_loss       | -6.29e-05 |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0007    |
|    n_updates          | 41699     |
|    policy_loss        | -1.57e-07 |
|    value_loss         | 0.0019    |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 3.32e+04  |
|    ep_rew_mean        | -1.43e+04 |
| time/                 |           |
|    fps                | 264       |
|    iterations         | 41800     |
|    time_elapsed       | 790       |
|    total_timesteps    | 209000    |
| train/    

In [ ]:
### Through this we evaluate our policy and observe we evaluation shall be proceeded with what level.
from stable_baselines3.common.evaluation import evaluate_policy
evaluate_policy(model, env, n_eval_episodes=20)

In [ ]:
A2C_path = os.path.join("Training","Saved Models","A2C_Model_MultiInputPolicy_long_data")

In [ ]:
model.save(PPO_path)